In [29]:
from pyspark.sql.functions import concat, lit, col
from pyspark.sql import SparkSession

In [30]:
spark = SparkSession.builder.master("local").appName("flights").getOrCreate()

In [31]:
df = spark.read.option("header",True).csv("./flights.csv")
df.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [33]:
df = df.filter("ARRIVAL_DELAY > 0")

In [34]:
df = df.drop("YEAR","MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","TAIL_NUMBER","SCHEDULED_DEPARTURE","DEPARTURE_TIME","DEPARTURE_DELAY","TAXI_OUT","WHEELS_OFF","SCHEDULED_TIME","ELAPSED_TIME","AIR_TIME","DISTANCE","WHEELS_ON","TAXI_IN","SCHEDULED_ARRIVAL","ARRIVAL_TIME","DIVERTED","CANCELLED","CANCELLATION_REASON","AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY")

In [35]:
df = df.withColumn('Ruta', concat(col('ORIGIN_AIRPORT'), col('DESTINATION_AIRPORT')))

In [36]:
df.show()

+--------------+-------------------+-------------+------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|ARRIVAL_DELAY|  Ruta|
+--------------+-------------------+-------------+------+
|           SFO|                CLT|            5|SFOCLT|
|           SFO|                MSP|            8|SFOMSP|
|           SEA|                MSP|            8|SEAMSP|
|           LAS|                MCI|            6|LASMCI|
|           SEA|                MIA|            1|SEAMIA|
|           MSP|                FLL|           25|MSPFLL|
|           DEN|                MIA|            2|DENMIA|
|           PHX|                ORD|           43|PHXORD|
|           SLC|                ATL|           10|SLCATL|
|           LAS|                HNL|           15|LASHNL|
|           SJU|                JFK|           11|SJUJFK|
|           PHX|                DFW|            1|PHXDFW|
|           SJU|                EWR|            4|SJUEWR|
|           BQN|                EWR|            6|BQNEWR|
|           BQ

Cantidad de Vuelos con retraso en el Aeropuerto de Atltanta

In [45]:
df.filter(df.DESTINATION_AIRPORT == "ATL").count()

105603

Las 10 rutas de vuelo más populares

In [48]:
df.groupBy('Ruta', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT').count().orderBy(col("Count").desc()).limit(10).show()

+------+--------------+-------------------+-----+
|  Ruta|ORIGIN_AIRPORT|DESTINATION_AIRPORT|count|
+------+--------------+-------------------+-----+
|SFOLAX|           SFO|                LAX| 6431|
|LAXSFO|           LAX|                SFO| 5845|
|LASLAX|           LAS|                LAX| 4974|
|LAXLAS|           LAX|                LAS| 4195|
|LAXJFK|           LAX|                JFK| 3972|
|JFKLAX|           JFK|                LAX| 3857|
|OGGHNL|           OGG|                HNL| 3765|
|ORDLGA|           ORD|                LGA| 3640|
|LAXPHX|           LAX|                PHX| 3567|
|ORDLAX|           ORD|                LAX| 3515|
+------+--------------+-------------------+-----+

